In [1]:
from transforms import NormalizeShift
import json
import pdb
from os import listdir
from os.path import isfile, join
import numpy as np
import pytorch3d
from pytorch3d.io import load_obj
import torch



In [5]:
def convert_cloud(file, i):
    try:
        bin_file_name = f'{i:06}.bin'
        point_cloud = pytorch3d.io.load_obj(file)
        np_point_cloud = point_cloud[0]
        norm_shift = NormalizeShift()
        np_point_cloud = norm_shift.fit_transform(np_point_cloud)
        xarr = np_point_cloud[:,0]
        yarr = np_point_cloud[:,1]
        zarr = np_point_cloud[:,2]
        ## Get data from pcd (x, y, z, intensity, ring, time)
        np_x = (np.array(xarr, dtype=np.float32)).astype(np.float32)
        np_y = (np.array(yarr, dtype=np.float32)).astype(np.float32)
        np_z = (np.array(zarr, dtype=np.float32)).astype(np.float32)
        np_i = (np.ones(len(zarr), dtype=np.float32)).astype(np.float32)
        transformed_cloud = np.transpose(np.vstack((np_x, np_y, np_z, np_i)))
        transformed_cloud.tofile('/multiview/3d-count/obj_detection/normalized_data/training/velodyne/' + bin_file_name)
        i += 1
    except:
        print("something went wrong")
    return norm_shift


In [6]:
def convert_label(file, i, norm_shift):
    try:
        txt_file_name = f'{i:06}.txt'
        with open('/multiview/3d-count/obj_detection/normalized_data/training/label_2/' + txt_file_name, "w") as text_file:
            label = json.load(open(file))
            bbs = label['bounding_boxes']
            for cls, points in bbs.items():
                class_name = cls.split("_")[0]
                occluded = 0
                truncated = 0
                alpha = 0
                points = norm_shift(torch.tensor(points))
                points = np.array(points)
                x = points[:,0]
                y = points[:,1]
                z = points[:,2]
                width = max(x) - min(x)
                height = max(y) - min(y)
                length = max(z) - min(z)
                location_x = np.mean(x)
                location_y = np.mean(y)
                location_z = np.mean(z)
                out_arr = list([class_name, str(float(truncated)), str(int(occluded)), str(float(alpha)), str(0.0), str(0.0), str(0.0), str(0.0), str(height), str(width), str(length), str(location_x), str(location_y), str(location_z)])
                out_str = " ".join(out_arr) + '\n'
                text_file.write(out_str)
            i += 1
    except:
        print("something went wrong")

In [7]:
metadata_file = '/multiview/3d-count/frustum-cooler-24k/train-metadata.json'
metadata = json.load(open(metadata_file))
counter = 0
for scenes in metadata.values():
    pc_file = scenes['point_cloud']
    label = scenes['label']
    norm = convert_cloud(pc_file,counter)
    convert_label(label, counter, norm)
    counter = counter + 1
    


In [108]:
metadata_file = '/multiview/3d-count/frustum-cooler-24k/test-metadata.json'
metadata = json.load(open(metadata_file))
file = metadata['4114ad3a-09ea-49c6-8ee8-5758fb50c66c-0']['point_cloud']
point_cloud = pytorch3d.io.load_obj(file)[0]
point_cloud

tensor([[-0.1716,  0.4954, -0.0554],
        [-0.1445,  0.5588, -0.0551],
        [-0.2099,  0.6360, -0.0567],
        ...,
        [-0.1424,  0.4667,  0.1196],
        [-0.1453,  0.4966, -0.2937],
        [-0.1513,  0.5729, -0.0733]])

In [54]:
norm_shift = NormalizeShift()
point_cloud = norm_shift.fit_transform(point_cloud)
xarr = point_cloud[:,0]
yarr = point_cloud[:,1]
zarr = point_cloud[:,2]
## Get data from pcd (x, y, z, intensity, ring, time)
np_x = (np.array(xarr, dtype=np.float32)).astype(np.float32)
np_y = (np.array(yarr, dtype=np.float32)).astype(np.float32)
np_z = (np.array(zarr, dtype=np.float32)).astype(np.float32)
np_i = (np.ones(len(zarr), dtype=np.float32)).astype(np.float32)
transformed_cloud = np.transpose(np.vstack((np_x, np_y, np_z, np_i)))
transformed_cloud.tofile('/multiview/3d-count/obj_detection/normalized_data/testing/velodyne/' + '000000.bin')

None
